In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
import pandas_datareader.data as web
import re

In [2]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By

# Find all links to fund info 

driver = webdriver.Chrome('/mnt/c/Users/bryan/OneDrive/Desktop/chromedriver/chromedriver.exe')
# driver.get('https://fund.cnyes.com/nav.aspx')

# 所有元大基金
# driver.get('http://fund.cdn.cnyes.com/nav.aspx?n=1&fundgroup=-%E5%9F%BA%E9%87%91%E9%A1%9E%E5%9E%8B-&CategoryLocal=-%E5%9F%BA%E9%87%91%E7%B5%84%E5%88%A5-&AllRegion=-%E6%8A%95%E8%B3%87%E5%8D%80%E5%9F%9F-&FundCompanyNameLocal=%E5%85%83%E5%A4%A7%E6%8A%95%E4%BF%A1&orderby=&fund=fundall&ddlindex=0,0,0,29')


# 所有野村基金
driver.get('http://fund.cdn.cnyes.com/nav.aspx?n=1&fundgroup=-%e5%9f%ba%e9%87%91%e9%a1%9e%e5%9e%8b-&CategoryLocal=-%e5%9f%ba%e9%87%91%e7%b5%84%e5%88%a5-&AllRegion=-%e6%8a%95%e8%b3%87%e5%8d%80%e5%9f%9f-&FundCompanyNameLocal=%e9%87%8e%e6%9d%91%e6%8a%95%e4%bf%a1&orderby=&fund=fundall&ddlindex=0,0,0,42')


time.sleep(2)

links = []

for i in range(10):
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    for item in soup.find_all('a'):
        link = item.get('href')
        if 'detail' in str(link) and link not in links:
            links.append(link)
    try:
        next_page = driver.find_element_by_id('example_next')
        next_page.click()
    except:
        break
        
funds = [x.split('/')[2] for x in links]

In [3]:
len(funds)

176

In [ ]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By

all_funds = []
driver = webdriver.Chrome('/mnt/c/Users/bryan/OneDrive/Desktop/chromedriver/chromedriver.exe')


for i in range(len(funds)):
    try:
        name = funds[i]
        print(i + 1, name)
        link = 'https://fund.cnyes.com' + links[i]
        driver.get(link)
        time.sleep(3)

        content = []
        current = '1'
        while True:
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")
            dates = []
            for info in soup.find_all('td', {'class':'_1JRsZ'}):
                dates.append(info.text)
                current = info.text


            nav = []
            for info in soup.find_all('td', {'class':False})[-30:]:
                if info.text != '0.00%' and info.text != '0.0000' and len(info.text)>0 and info.text[-1] != '%':
        #             print(info.text)
                    nav.append(info.text)

            content.append(pd.DataFrame({'Date':dates, name:nav}))

            try:
                next_page = driver.find_element_by_xpath("//a[contains(text(),'下一頁')]")
                next_page.click()
            except:
                break

            if '2016/01/04' in dates:
                break

        final = pd.concat(content)
        final.drop_duplicates(inplace = True)
        final.sort_values(['Date'], inplace = True)
        final.set_index(['Date'], inplace = True)
        all_funds.append(final)
        save = pd.concat(all_funds, axis = 1, sort = True)
        save.to_csv('野村基金淨值.csv')
    except:
        continue
    
    
save

1 野村ｅ科技基金
2 野村高科技基金
3 野村中小基金-累積類型
4 野村優質基金-累積類型新臺幣計價
5 野村優質基金-S類型新臺幣累積
6 野村台灣運籌基金
7 野村鴻運基金
8 野村積極成長基金
